In [14]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import talib as ta
import importlib
import os
import platform
import psutil

In [15]:
#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html

# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-1)
end = datetime.date.today()
# data = web.DataReader('7203.T', 'yahoo', start, end)
# data = web.DataReader('SPYD', 'yahoo', start, end)
# data = web.DataReader('1321.T', 'yahoo', start, end)
# data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
# data = web.DataReader('^N225', 'yahoo', start, end)
# data = web.DataReader('AAPL', 'yahoo', start, end)
data = data.astype("double")
print(data)

initial = 1000000
fname = f"html/plot-"

#position
#0 ... display "only long" result
#1 ... display both "only long" and "long and short" result
position = 1

             High    Low   Open  Close       Volume  Adj Close
Date                                                          
2021-09-13  370.0  364.0  367.0  365.0   33019794.0      365.0
2021-09-14  363.0  356.0  361.0  358.0   66934700.0      358.0
2021-09-15  367.0  361.0  364.0  362.0   54868371.0      362.0
2021-09-16  371.0  360.0  360.0  367.0   52270565.0      367.0
2021-09-17  367.0  362.0  366.0  362.0   43735573.0      362.0
...           ...    ...    ...    ...          ...        ...
2022-09-07  380.0  371.0  372.0  374.0  110425487.0      374.0
2022-09-08  367.0  357.0  367.0  357.0  113510594.0      357.0
2022-09-09  357.0  352.0  354.0  353.0   99795902.0      353.0
2022-09-12  348.0  343.0  346.0  346.0   73706891.0      346.0
2022-09-13  346.0  342.0  345.0  344.0   29228803.0      344.0

[246 rows x 6 columns]


**MACD CROSS**

In [23]:
import BacktestingStrategies.Strategy_MacdCross as btstMACD
importlib.reload(btstMACD)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACD.MACDCross, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）

#最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)
output2=bt.optimize(\
    n1=range(10, 25, 5),\
    n2=range(10, 75, 10),\
    ns=range(10, 15, 5),\
    maximize='Equity Final [$]',
    method='grid',\
    constraint = lambda x: (x.n2 > x.n1) & (x.n1 > x.ns),\
    max_tries = 100)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 


if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACD.MACDCross_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True, #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）

    #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)
    output2=bt.optimize(\
        n1=range(10, 25, 5),\
        n2=range(10, 75, 10),\
        ns=range(10, 15, 5),\
        maximize='Equity Final [$]',
        method='grid',\
        constraint = lambda x: (x.n2 > x.n1) & (x.n1 > x.ns),\
        max_tries = 100)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 


Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   41.869919
Equity Final [$]                  947049.1966
Equity Peak [$]                 1156303.13075
Return [%]                           -5.29508
Buy & Hold Return [%]               -5.753425
Return (Ann.) [%]                   -5.420664
Volatility (Ann.) [%]               26.680506
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -18.333509
Avg. Drawdown [%]                   -10.68904
Max. Drawdown Duration      188 days 00:00:00
Avg. Drawdown Duration       72 days 00:00:00
# Trades                                    7
Win Rate [%]                        28.571429
Best Trade [%]                       3.985273
Worst Trade [%]                      -5.60743
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2519,50,59,396.95525,391.0,-15001.27475,-0.015002,2021-11-26,2021-12-09,13 days
1,2450,79,97,401.98000,418.0,39249.00000,0.039853,2022-01-11,2022-02-04,24 days
2,2285,108,123,448.20770,462.0,31515.40550,0.030772,2022-02-22,2022-03-16,22 days
3,2574,139,163,410.01960,406.0,-10346.45040,-0.009803,2022-04-08,2022-05-18,40 days
4,2543,182,193,411.02455,401.0,-25492.43065,-0.024389,2022-06-14,2022-06-29,15 days
5,2371,195,200,430.11860,406.0,-57185.20060,-0.056074,2022-07-01,2022-07-08,7 days
6,2737,230,244,351.73250,346.0,-15689.85250,-0.016298,2022-08-23,2022-09-12,20 days


Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   43.902439
Equity Final [$]                  930752.8923
Equity Peak [$]                 1202948.79715
Return [%]                          -6.924711
Buy & Hold Return [%]               -5.753425
Return (Ann.) [%]                   -7.087476
Volatility (Ann.) [%]               27.592238
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -22.851006
Avg. Drawdown [%]                    -10.6419
Max. Drawdown Duration      188 days 00:00:00
Avg. Drawdown Duration       71 days 00:00:00
# Trades                                    6
Win Rate [%]                        16.666667
Best Trade [%]                       7.630494
Worst Trade [%]                     -4.285446
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2519,50,61,396.95525,393.0,-9963.27475,-0.009964,2021-11-26,2021-12-13,17 days
1,2513,81,98,393.94040,424.0,75539.77480,0.076305,2022-01-13,2022-02-07,25 days
2,2377,108,124,448.20770,429.0,-45656.70290,-0.042854,2022-02-22,2022-03-17,23 days
3,2451,140,166,416.04930,401.0,-36885.83430,-0.036172,2022-04-11,2022-05-23,42 days
4,2391,182,201,411.02455,399.0,-28750.69905,-0.029255,2022-06-14,2022-07-11,27 days
5,2690,231,244,354.74735,346.0,-23530.37150,-0.024658,2022-08-24,2022-09-12,19 days


<Strategy MACDCross(n1=15,n2=30,ns=10)>

Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   79.268293
Equity Final [$]                 987086.76455
Equity Peak [$]                  1149768.8413
Return [%]                          -1.291324
Buy & Hold Return [%]               -5.753425
Return (Ann.) [%]                    -1.32261
Volatility (Ann.) [%]               37.113448
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -24.984438
Avg. Drawdown [%]                  -11.542774
Max. Drawdown Duration       96 days 00:00:00
Avg. Drawdown Duration       41 days 00:00:00
# Trades                                   13
Win Rate [%]                        30.769231
Best Trade [%]                      13.364257
Worst Trade [%]                     -7.264124
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2519,50,59,396.95525,391.0,-15001.27475,-0.015002,2021-11-26,2021-12-09,13 days
1,-2531,59,79,389.06455,400.0,-27677.62395,-0.028107,2021-12-09,2022-01-11,33 days
2,2381,79,97,401.98000,418.0,38143.62000,0.039853,2022-01-11,2022-02-04,24 days
3,-2393,97,108,415.93090,446.0,-71955.35630,-0.072293,2022-02-04,2022-02-22,18 days
4,2060,108,123,448.20770,462.0,28412.13800,0.030772,2022-02-22,2022-03-16,22 days
5,-2070,123,139,459.71310,408.0,107046.11700,0.112490,2022-03-16,2022-04-08,23 days
6,2582,139,163,410.01960,406.0,-10378.60720,-0.009803,2022-04-08,2022-05-18,40 days
7,-2595,163,182,403.99030,409.0,-13000.17150,-0.012401,2022-05-18,2022-06-14,27 days
8,2519,182,193,411.02455,401.0,-25251.84145,-0.024389,2022-06-14,2022-06-29,15 days
9,-2532,193,195,399.01505,428.0,-73389.89340,-0.072641,2022-06-29,2022-07-01,2 days


Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   79.268293
Equity Final [$]                  976786.7962
Equity Peak [$]                  1155762.4616
Return [%]                           -2.32132
Buy & Hold Return [%]               -5.753425
Return (Ann.) [%]                    -2.37726
Volatility (Ann.) [%]                 36.8673
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -25.329763
Avg. Drawdown [%]                  -12.573548
Max. Drawdown Duration      229 days 00:00:00
Avg. Drawdown Duration       95 days 00:00:00
# Trades                                   11
Win Rate [%]                        27.272727
Best Trade [%]                      11.088711
Worst Trade [%]                     -5.711953
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2519,50,61,396.95525,393.0,-9963.27475,-0.009964,2021-11-26,2021-12-13,17 days
1,-2531,61,81,391.05465,392.0,-2392.68085,-0.002417,2021-12-13,2022-01-13,31 days
2,2507,81,98,393.94040,424.0,75359.41720,0.076305,2022-01-13,2022-02-07,25 days
3,-2519,98,108,421.90120,446.0,-60704.87720,-0.057120,2022-02-07,2022-02-22,15 days
4,2236,108,124,448.20770,429.0,-42948.41720,-0.042854,2022-02-22,2022-03-17,23 days
5,-2247,124,140,426.87645,414.0,28933.38315,0.030164,2022-03-17,2022-04-11,25 days
6,2375,140,166,416.04930,401.0,-35742.08750,-0.036172,2022-04-11,2022-05-23,42 days
7,-2387,166,182,399.01505,409.0,-23834.07565,-0.025024,2022-05-23,2022-06-14,22 days
8,2259,182,201,411.02455,399.0,-27163.45845,-0.029255,2022-06-14,2022-07-11,27 days
9,-2270,201,231,397.02495,353.0,99936.63650,0.110887,2022-07-11,2022-08-24,44 days


<Strategy MACDCross_WithShortPosition(n1=15,n2=30,ns=10)>

**EMA CROSS**

In [24]:
import BacktestingStrategies.Strategy_EMACross as btstEMA
importlib.reload(btstEMA)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstEMA.EmaCrossStrategy, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html") 

#最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    n1=range(10, 100, 10),\
    n2=range(10, 200, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    max_tries = 100)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstEMA.EmaCrossStrategy_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") 

    #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        n1=range(10, 100, 10),\
        n2=range(10, 200, 5),\
        maximize='Equity Final [$]',\
        method='grid',\
        max_tries = 100)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   41.869919
Equity Final [$]                 770878.05475
Equity Peak [$]                  1098726.1086
Return [%]                         -22.912195
Buy & Hold Return [%]               -5.753425
Return (Ann.) [%]                  -23.399918
Volatility (Ann.) [%]               22.271738
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -29.838924
Avg. Drawdown [%]                  -14.970146
Max. Drawdown Duration      188 days 00:00:00
Avg. Drawdown Duration       72 days 00:00:00
# Trades                                   10
Win Rate [%]                             20.0
Best Trade [%]                       3.985273
Worst Trade [%]                      -5.60743
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2519,50,59,396.95525,391.0,-15001.27475,-0.015002,2021-11-26,2021-12-09,13 days
1,2378,66,68,414.03940,394.0,-47653.69320,-0.048400,2021-12-20,2021-12-22,2 days
2,2331,79,97,401.98000,418.0,37342.62000,0.039853,2022-01-11,2022-02-04,24 days
3,2234,103,104,436.14830,415.0,-47245.30220,-0.048489,2022-02-15,2022-02-16,1 days
4,2171,106,124,427.10375,429.0,4116.75875,0.004440,2022-02-18,2022-03-17,27 days
5,2271,139,147,410.01960,400.0,-22754.51160,-0.024437,2022-04-08,2022-04-20,12 days
6,2163,150,163,420.06910,406.0,-30431.46330,-0.033492,2022-04-25,2022-05-18,23 days
7,2137,182,192,411.02455,396.0,-32107.46335,-0.036554,2022-06-14,2022-06-28,14 days
8,1967,195,200,430.11860,406.0,-47441.28620,-0.056074,2022-07-01,2022-07-08,7 days
9,2183,234,243,365.80180,353.0,-27946.32940,-0.034997,2022-08-29,2022-09-09,11 days


Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   42.276423
Equity Final [$]                1249871.98685
Equity Peak [$]                 1448441.86745
Return [%]                          24.987199
Buy & Hold Return [%]               -5.753425
Return (Ann.) [%]                   25.668985
Volatility (Ann.) [%]               30.179446
Sharpe Ratio                         0.850545
Sortino Ratio                        1.702123
Calmar Ratio                         1.492348
Max. Drawdown [%]                  -17.200406
Avg. Drawdown [%]                   -4.142451
Max. Drawdown Duration       60 days 00:00:00
Avg. Drawdown Duration       13 days 00:00:00
# Trades                                    6
Win Rate [%]                        83.333333
Best Trade [%]                       15.70025
Worst Trade [%]                    -13.710342
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2667,75,79,374.84635,400.0,67084.78455,0.067104,2022-01-04,2022-01-11,7 days
1,2765,80,82,385.90080,402.0,44514.28800,0.041718,2022-01-12,2022-01-14,2 days
2,2903,131,159,382.88595,443.0,174511.08715,0.157002,2022-03-29,2022-05-12,44 days
3,3191,166,188,402.98495,422.0,60677.02455,0.047186,2022-05-23,2022-06-22,30 days
4,3384,192,195,397.96020,428.0,101654.68320,0.075484,2022-06-28,2022-07-01,3 days
5,3612,205,244,400.97505,346.0,-198569.88060,-0.137103,2022-07-15,2022-09-12,59 days


<Strategy EmaCrossStrategy(n1=70,n2=15)>

Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   90.650407
Equity Final [$]                 647208.83325
Equity Peak [$]                   1051699.032
Return [%]                         -35.279117
Buy & Hold Return [%]               -5.753425
Return (Ann.) [%]                  -35.962294
Volatility (Ann.) [%]               25.284347
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -44.243048
Avg. Drawdown [%]                  -10.850161
Max. Drawdown Duration      301 days 00:00:00
Avg. Drawdown Duration       67 days 00:00:00
# Trades                                   21
Win Rate [%]                        23.809524
Best Trade [%]                       9.898827
Worst Trade [%]                      -8.61847
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-2576,22,50,388.06950,395.0,-17852.96800,-0.017859,2021-10-15,2021-11-26,42 days
1,2474,50,59,396.95525,391.0,-14733.28850,-0.015002,2021-11-26,2021-12-09,13 days
2,-2486,59,66,389.06455,412.0,-57017.52870,-0.058950,2021-12-09,2021-12-20,11 days
3,2198,66,68,414.03940,394.0,-44046.60120,-0.048400,2021-12-20,2021-12-22,2 days
4,-2209,68,79,392.04970,400.0,-17562.21270,-0.020279,2021-12-22,2022-01-11,20 days
5,2111,79,97,401.98000,418.0,33818.22000,0.039853,2022-01-11,2022-02-04,24 days
6,-2122,97,103,415.93090,434.0,-38342.63020,-0.043443,2022-02-04,2022-02-15,11 days
7,1935,103,104,436.14830,415.0,-40921.96050,-0.048489,2022-02-15,2022-02-16,1 days
8,-1945,104,106,412.94575,425.0,-23445.51625,-0.029191,2022-02-16,2022-02-18,2 days
9,1826,106,124,427.10375,429.0,3462.55250,0.004440,2022-02-18,2022-03-17,27 days


Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   69.105691
Equity Final [$]                1660404.20425
Equity Peak [$]                 1924174.49415
Return [%]                           66.04042
Buy & Hold Return [%]               -5.753425
Return (Ann.) [%]                    68.10665
Volatility (Ann.) [%]               68.699012
Sharpe Ratio                         0.991377
Sortino Ratio                        2.753917
Calmar Ratio                         2.502154
Max. Drawdown [%]                  -27.219211
Avg. Drawdown [%]                   -5.158195
Max. Drawdown Duration       65 days 00:00:00
Avg. Drawdown Duration       14 days 00:00:00
# Trades                                   11
Win Rate [%]                        90.909091
Best Trade [%]                       15.70025
Worst Trade [%]                    -13.710342
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2667,75,79,374.84635,400.0,67084.78455,0.067104,2022-01-04,2022-01-11,7 days
1,-2680,79,80,398.02000,384.0,37573.60000,0.035224,2022-01-11,2022-01-12,1 days
2,2862,80,82,385.90080,402.0,46075.91040,0.041718,2022-01-12,2022-01-14,2 days
3,-2876,82,131,400.01010,381.0,54673.04760,0.047524,2022-01-14,2022-03-29,74 days
4,3148,131,159,382.88595,443.0,189239.02940,0.157002,2022-03-29,2022-05-12,44 days
5,-3163,159,166,440.80715,401.0,125910.01545,0.090305,2022-05-12,2022-05-23,11 days
6,3773,166,188,402.98495,422.0,71743.78365,0.047186,2022-05-23,2022-06-22,30 days
7,-3791,188,192,419.91110,396.0,90646.98010,0.056943,2022-06-22,2022-06-28,6 days
8,4228,192,195,397.96020,428.0,127008.27440,0.075484,2022-06-28,2022-07-01,3 days
9,-4249,195,205,425.88140,399.0,114219.06860,0.063119,2022-07-01,2022-07-15,14 days


<Strategy EmaCrossStrategy_WithShortPosition(n1=70,n2=15)>

**SMA**

In [25]:
import BacktestingStrategies.Strategy_SMACross as btstSMA
importlib.reload(btstSMA)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstSMA.SmaCross, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    n1=range(10, 100, 5),\
    n2=range(10, 200, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    max_tries = 100)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstSMA.SmaCross_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        n1=range(10, 100, 5),\
        n2=range(10, 200, 5),\
        maximize='Equity Final [$]',\
        method='grid',\
        max_tries = 100)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   44.308943
Equity Final [$]                 666943.56785
Equity Peak [$]                  1084176.5727
Return [%]                         -33.305643
Buy & Hold Return [%]               -5.753425
Return (Ann.) [%]                   -33.96129
Volatility (Ann.) [%]               20.071179
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -38.483861
Avg. Drawdown [%]                  -25.624292
Max. Drawdown Duration      188 days 00:00:00
Avg. Drawdown Duration      142 days 00:00:00
# Trades                                    5
Win Rate [%]                              0.0
Best Trade [%]                      -0.973274
Worst Trade [%]                    -12.282543
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2392,53,72,418.05920,379.0,-93429.60640,-0.093430,2021-12-01,2021-12-28,27 days
1,2142,85,128,423.08395,384.0,-83717.82090,-0.092379,2022-01-19,2022-03-24,64 days
2,1977,151,169,416.04930,412.0,-8005.46610,-0.009733,2022-04-26,2022-05-26,30 days
3,1921,188,207,424.08890,372.0,-100062.77690,-0.122825,2022-06-22,2022-07-20,28 days
4,1927,239,244,370.82655,346.0,-47840.76185,-0.066949,2022-09-05,2022-09-12,7 days


Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   46.747967
Equity Final [$]                1242862.37915
Equity Peak [$]                 1454650.00555
Return [%]                          24.286238
Buy & Hold Return [%]               -5.753425
Return (Ann.) [%]                    24.94706
Volatility (Ann.) [%]                30.90669
Sharpe Ratio                         0.807173
Sortino Ratio                        1.654884
Calmar Ratio                         1.384686
Max. Drawdown [%]                  -18.016404
Avg. Drawdown [%]                   -4.058479
Max. Drawdown Duration       85 days 00:00:00
Avg. Drawdown Duration       25 days 00:00:00
# Trades                                    5
Win Rate [%]                             80.0
Best Trade [%]                      11.103156
Worst Trade [%]                     -5.930127
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2696,35,55,370.82655,412.0,111003.62120,0.111032,2021-11-04,2021-12-03,29 days
1,2963,75,88,374.84635,416.0,121938.26495,0.109788,2022-01-04,2022-01-24,20 days
2,3220,132,155,382.88595,404.0,67987.24100,0.055144,2022-03-30,2022-05-06,37 days
3,3336,173,192,389.92060,396.0,20280.87840,0.015591,2022-06-01,2022-06-28,27 days
4,3592,209,244,367.81170,346.0,-78347.62640,-0.059301,2022-07-22,2022-09-12,52 days


<Strategy SmaCross(n1=35,n2=15)>

Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                    86.99187
Equity Final [$]                  495279.4879
Equity Peak [$]                   1063713.844
Return [%]                         -50.472051
Buy & Hold Return [%]               -5.753425
Return (Ann.) [%]                  -51.313599
Volatility (Ann.) [%]               19.432442
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -53.438654
Avg. Drawdown [%]                  -19.493348
Max. Drawdown Duration      301 days 00:00:00
Avg. Drawdown Duration      105 days 00:00:00
# Trades                                   10
Win Rate [%]                             10.0
Best Trade [%]                       0.313001
Worst Trade [%]                    -12.282543
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-2544,31,53,393.04475,416.0,-58398.15600,-0.058404,2021-10-28,2021-12-01,34 days
1,2252,53,72,418.05920,379.0,-87961.31840,-0.093430,2021-12-01,2021-12-28,27 days
2,-2263,72,85,377.12395,421.0,-99291.50115,-0.116344,2021-12-28,2022-01-19,22 days
3,1782,85,128,423.08395,384.0,-69647.59890,-0.092379,2022-01-19,2022-03-24,64 days
4,-1791,128,151,382.09920,414.0,-57134.33280,-0.083488,2022-03-24,2022-04-26,33 days
5,1508,151,169,416.04930,412.0,-6106.34440,-0.009733,2022-04-26,2022-05-26,30 days
6,-1515,169,188,409.96060,422.0,-18239.69100,-0.029367,2022-05-26,2022-06-22,27 days
7,1422,188,207,424.08890,372.0,-74070.41580,-0.122825,2022-06-22,2022-07-20,28 days
8,-1429,207,239,370.15860,369.0,1655.63940,0.003130,2022-07-20,2022-09-05,47 days
9,1431,239,244,370.82655,346.0,-35526.79305,-0.066949,2022-09-05,2022-09-12,7 days


Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   86.585366
Equity Final [$]                1353604.58855
Equity Peak [$]                  1445176.8507
Return [%]                          35.360459
Buy & Hold Return [%]               -5.753425
Return (Ann.) [%]                    36.36375
Volatility (Ann.) [%]               57.895413
Sharpe Ratio                         0.628094
Sortino Ratio                        1.373023
Calmar Ratio                         1.595847
Max. Drawdown [%]                   -22.78649
Avg. Drawdown [%]                   -6.385012
Max. Drawdown Duration      154 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                                   12
Win Rate [%]                        66.666667
Best Trade [%]                      11.571517
Worst Trade [%]                     -7.653147
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2531,32,53,394.94535,416.0,53289.31915,0.053310,2021-10-29,2021-12-01,33 days
1,-2544,53,70,413.94080,387.0,68537.39520,0.065084,2021-12-01,2021-12-24,23 days
2,2884,70,87,388.91565,418.0,83879.26540,0.074783,2021-12-24,2022-01-21,28 days
3,-2898,87,107,415.93090,432.0,-46568.25180,-0.038634,2022-01-21,2022-02-21,31 days
4,2669,107,112,434.13840,433.0,-3038.38960,-0.002622,2022-02-21,2022-03-01,8 days
5,-2683,112,131,430.85665,381.0,133765.39195,0.115715,2022-03-01,2022-03-29,28 days
6,3368,131,148,382.88595,390.0,23960.12040,0.018580,2022-03-29,2022-04-21,23 days
7,-3385,148,172,388.06950,393.0,-16689.74250,-0.012705,2022-04-21,2022-05-31,40 days
8,3284,172,190,394.94535,411.0,52723.47060,0.040650,2022-05-31,2022-06-24,24 days
9,-3300,190,204,408.96555,403.0,19686.31500,0.014587,2022-06-24,2022-07-14,20 days


<Strategy SmaCross_WithShortPosition(n1=25,n2=15)>

**ボリンジャーバンド**

In [26]:
import BacktestingStrategies.Strategy_BBD as btstBBD
importlib.reload(btstBBD)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstBBD.BBsigma, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstBBD.BBsigma_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")


Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   41.869919
Equity Final [$]                 1014803.7529
Equity Peak [$]                  1297779.4732
Return [%]                           1.480375
Buy & Hold Return [%]               -5.753425
Return (Ann.) [%]                    1.516754
Volatility (Ann.) [%]               24.087391
Sharpe Ratio                         0.062969
Sortino Ratio                        0.097332
Calmar Ratio                         0.060749
Max. Drawdown [%]                  -24.967549
Avg. Drawdown [%]                   -4.934518
Max. Drawdown Duration       85 days 00:00:00
Avg. Drawdown Duration       17 days 00:00:00
# Trades                                    7
Win Rate [%]                        28.571429
Best Trade [%]                      14.201542
Worst Trade [%]                     -5.930127
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2577,127,159,387.91070,443.0,141965.1261,0.142015,2022-03-23,2022-05-12,50 days
1,3006,175,176,379.87110,374.0,-17648.5266,-0.015456,2022-06-03,2022-06-06,3 days
2,2991,176,178,375.85130,365.0,-32456.2383,-0.028871,2022-06-06,2022-06-08,2 days
3,2976,178,207,366.80675,372.0,15455.1120,0.014158,2022-06-08,2022-07-20,42 days
4,2961,207,208,373.84140,370.0,-11374.3854,-0.010275,2022-07-20,2022-07-21,1 days
5,2947,208,209,371.83150,366.0,-17185.4305,-0.015683,2022-07-21,2022-07-22,1 days
6,2932,209,244,367.81170,346.0,-63951.9044,-0.059301,2022-07-22,2022-09-12,52 days


Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   78.861789
Equity Final [$]                1496493.09885
Equity Peak [$]                 1621918.09885
Return [%]                           49.64931
Buy & Hold Return [%]               -5.753425
Return (Ann.) [%]                   51.127965
Volatility (Ann.) [%]               63.671755
Sharpe Ratio                         0.802993
Sortino Ratio                        1.990865
Calmar Ratio                         1.662635
Max. Drawdown [%]                  -30.751174
Avg. Drawdown [%]                    -5.09867
Max. Drawdown Duration       92 days 00:00:00
Avg. Drawdown Duration       15 days 00:00:00
# Trades                                    6
Win Rate [%]                        83.333333
Best Trade [%]                      12.933045
Worst Trade [%]                     -7.447383
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-2469,51,127,404.98535,386.0,46874.82915,0.046879,2021-11-29,2022-03-23,114 days
1,2698,127,152,387.91070,425.0,100066.93140,0.095613,2022-03-23,2022-04-27,35 days
2,-2712,152,175,422.89625,378.0,121758.63000,0.106164,2022-04-27,2022-06-03,37 days
3,3339,175,185,379.87110,429.0,164041.39710,0.129330,2022-06-03,2022-06-17,14 days
4,-3356,185,207,426.87645,372.0,184165.36620,0.128553,2022-06-17,2022-07-20,33 days
5,4325,207,244,373.84140,346.0,-120414.05500,-0.074474,2022-07-20,2022-09-12,54 days


**RSI**

In [28]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    upper_bound = range(50, 85, 5),
    lower_bound = range(10, 45, 5),
    rsi_window=range(10, 30, 2),\
    maximize='Equity Final [$]',\
    method='grid',\
    max_tries = 100)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstRsi.RsiOscillator_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        upper_bound = range(50, 85, 5),
        lower_bound = range(10, 45, 5),
        rsi_window=range(10, 30, 2),\
        maximize='Equity Final [$]',\
        method='grid',\
        max_tries = 100)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                    7.317073
Equity Final [$]                1015619.92325
Equity Peak [$]                 1097799.92325
Return [%]                           1.561992
Buy & Hold Return [%]               -5.753425
Return (Ann.) [%]                    1.600393
Volatility (Ann.) [%]                9.398069
Sharpe Ratio                          0.17029
Sortino Ratio                        0.267235
Calmar Ratio                         0.213788
Max. Drawdown [%]                   -7.485881
Avg. Drawdown [%]                   -2.597391
Max. Drawdown Duration        6 days 00:00:00
Avg. Drawdown Duration        5 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       1.562164
Worst Trade [%]                      1.562164
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2935,227,244,340.67805,346.0,15619.92325,0.015622,2022-08-18,2022-09-12,25 days


Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                    26.01626
Equity Final [$]                 1286810.7935
Equity Peak [$]                 1323674.03775
Return [%]                          28.681079
Buy & Hold Return [%]               -5.753425
Return (Ann.) [%]                    29.47496
Volatility (Ann.) [%]               24.188478
Sharpe Ratio                         1.218554
Sortino Ratio                        2.747042
Calmar Ratio                         2.389332
Max. Drawdown [%]                  -12.336069
Avg. Drawdown [%]                    -3.34053
Max. Drawdown Duration       42 days 00:00:00
Avg. Drawdown Duration       11 days 00:00:00
# Trades                                   10
Win Rate [%]                             70.0
Best Trade [%]                       9.384107
Worst Trade [%]                     -5.101898
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2646,37,44,377.86120,371.0,-18154.73520,-0.018158,2021-11-08,2021-11-17,9 days
1,2633,44,48,372.83645,380.0,18861.62715,0.019214,2021-11-17,2021-11-24,7 days
2,2620,48,50,381.88100,395.0,34371.78000,0.034354,2021-11-24,2021-11-26,2 days
3,2620,77,85,394.94535,421.0,68263.18300,0.065970,2022-01-06,2022-01-19,13 days
4,2765,137,141,398.96515,429.0,83046.36025,0.075282,2022-04-06,2022-04-12,6 days
5,2929,149,159,404.99485,443.0,111317.08435,0.093841,2022-04-22,2022-05-12,20 days
6,3236,181,182,400.97505,409.0,25968.73820,0.020014,2022-06-13,2022-06-14,1 days
7,3512,216,222,376.85625,367.0,-34615.15000,-0.026154,2022-08-02,2022-08-10,8 days
8,3495,222,230,368.81665,350.0,-65764.19175,-0.051019,2022-08-10,2022-08-23,13 days
9,3477,230,238,351.73250,370.0,63516.09750,0.051936,2022-08-23,2022-09-02,10 days


<Strategy RsiOscillator(upper_bound=60,lower_bound=40,rsi_window=10)>

Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   78.861789
Equity Final [$]                 1423902.0636
Equity Peak [$]                  1543237.0636
Return [%]                          42.390206
Buy & Hold Return [%]               -5.753425
Return (Ann.) [%]                   43.622849
Volatility (Ann.) [%]                60.39395
Sharpe Ratio                         0.722305
Sortino Ratio                        1.670043
Calmar Ratio                         1.418575
Max. Drawdown [%]                  -30.751174
Avg. Drawdown [%]                   -5.043233
Max. Drawdown Duration      128 days 00:00:00
Avg. Drawdown Duration       18 days 00:00:00
# Trades                                    6
Win Rate [%]                        66.666667
Best Trade [%]                      14.248215
Worst Trade [%]                     -7.447383
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-2469,51,126,404.98535,411.0,-14850.17085,-0.014852,2021-11-29,2022-03-22,113 days
1,2385,126,159,413.03445,443.0,71467.83675,0.072550,2022-03-22,2022-05-12,51 days
2,-2397,159,175,440.80715,378.0,150548.73855,0.142482,2022-05-12,2022-06-03,22 days
3,3177,175,185,379.87110,429.0,156082.51530,0.129330,2022-06-03,2022-06-17,14 days
4,-3193,185,207,426.87645,372.0,175220.50485,0.128553,2022-06-17,2022-07-20,33 days
5,4115,207,244,373.84140,346.0,-114567.36100,-0.074474,2022-07-20,2022-09-12,54 days


Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   91.869919
Equity Final [$]                 2171271.8522
Equity Peak [$]                  2182019.0447
Return [%]                         117.127185
Buy & Hold Return [%]               -5.753425
Return (Ann.) [%]                  121.272144
Volatility (Ann.) [%]               94.572156
Sharpe Ratio                         1.282324
Sortino Ratio                        4.855022
Calmar Ratio                          5.36244
Max. Drawdown [%]                  -22.615103
Avg. Drawdown [%]                    -4.63417
Max. Drawdown Duration       63 days 00:00:00
Avg. Drawdown Duration       12 days 00:00:00
# Trades                                   14
Win Rate [%]                        71.428571
Best Trade [%]                      13.030244
Worst Trade [%]                     -2.407153
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-2415,19,33,413.94080,372.0,101287.03200,0.101321,2021-10-12,2021-11-01,20 days
1,2945,33,50,373.84140,395.0,62312.07700,0.056598,2021-11-01,2021-11-26,25 days
2,-2960,50,75,393.04475,373.0,59332.46000,0.050999,2021-11-26,2022-01-04,39 days
3,3262,75,85,374.84635,421.0,150553.20630,0.123127,2022-01-04,2022-01-19,15 days
4,-3278,85,124,418.91605,429.0,-33055.18810,-0.024072,2022-01-19,2022-03-17,57 days
5,3109,124,141,431.12355,429.0,-6602.11695,-0.004926,2022-03-17,2022-04-12,26 days
6,-3124,141,148,426.87645,390.0,115202.02980,0.086387,2022-04-12,2022-04-21,9 days
7,3697,148,159,391.93050,443.0,188803.94150,0.130302,2022-04-21,2022-05-12,21 days
8,-3715,159,171,440.80715,389.0,192463.56225,0.117528,2022-05-12,2022-05-30,18 days
9,4681,171,182,390.92555,409.0,84606.50045,0.046235,2022-05-30,2022-06-14,15 days


<Strategy RsiOscillator_WithShortPosition(upper_bound=60,lower_bound=40,rsi_window=10)>